# 对JD商品评价进行分类

In [2]:
import numpy as np
import re
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [3]:
#每句话最大长度
MAX_LEN=64

In [11]:
#数据清理
def clean(sent):
    punctuation_remove = u'[、：，？！。；……（）『』《》【】～!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+'
    sent = re.sub(r'ldquo', "", sent)
    sent = re.sub(r'hellip', "", sent)
    sent = re.sub(r'rdquo', "", sent)
    sent = re.sub(r'yen', "", sent)
    sent = re.sub(r'⑦', "7", sent)
    sent = re.sub(r'(， ){2,}', "", sent)
    sent = re.sub(r'(！ ){2,}', "", sent)  # delete too many！，？，。等
    sent = re.sub(r'(？ ){2,}', "", sent)
    sent = re.sub(r'(。 ){2,}', "", sent)
    sent = re.sub(r'\n', "", sent)
    # sent=sent.split()
    # sent_no_pun=[]
    # for word in sent:
    #     if(word!=('，'or'。'or'？'or'！'or'：'or'；'or'（'or'）'or'『'or'』'or'《'or'》'or'【'or'】'or'～'or'!'or'\"'or'\''or'?'or','or'.')):
    #         sent_no_pun.append(word)
    # s=' '.join(sent_no_pun)
    sent = re.sub(punctuation_remove, "", sent)  # delete punctuations
    #若长度大于64，则截取前64个长度
    if(len(sent.split())>MAX_LEN):
        s=' '.join(sent.split()[:MAX_LEN])
    else:
        s = ' '.join(sent.split())  # delete additional space

    return s

In [12]:
good_data=open('D:/opt/JD_nlp/data/good_cut_jieba.txt','r',encoding='utf-8').readlines()

In [13]:
good_data

['整个 感觉 除了 送货 师傅 新手 来晚 了 外 ， 其他 都 很 好\n',
 '京东 自营 很 不错 ， ， ， ， ，\n',
 '外观 手感 使用 都 不错 ， 好 ！\n',
 '相信 这个 牌子 ， 没有 验证 过 保修期\n',
 '好 ， 售后服务 到位 ， 能 解决 使用 中 遇见 的 问题 。\n',
 '你 哦 ， 手机 用 着 挺 好 的 ， 用 了 好长时间 才 过来 评价 ， 挺不错 的 。\n',
 '给 老公 买 的       这 几天 用 的 挺 好 的       一直 在 京东 上 买 东西       很 可靠\n',
 '比 sSN 便宜 、 居然 SN 是 敏感 词语 。 真逗\n',
 '感觉 还好 ， 没 发现 什么 问题 。 快递 很快 ， 上午 下单 下午 就 到 。 就是 箱子 里面 没 放 气泡\n',
 '还 可以 的 ， 用 起来 很 流畅\n',
 '非常 好 ， 京东 配送 服务 好 ， 老婆 很 喜欢 ， 很 好 用 。\n',
 '查 了 不是 翻新机\n',
 '头 一次 买 苹果 ， 开机 激活 费 了 点 事儿 ， 不过 刚好 证明 机子 是 新 的 ， 用 着 还 挺 好 。\n',
 '啥 都 好 ， 就是 没 给 发票 ， 你们 是不是 给 忘 了\n',
 '及时 ， 速度 ， 产品质量 佳\n',
 '可以 可以 可以 不 可以 可以 不 可以 可以 不 可以 可以 不 可以\n',
 '手机 不错   快递员 超有 礼貌   拆 包装 时 一直 强调 小心   不止 五星 的 好评\n',
 '梦寐以求 的 6s 终于 到手 了 ， 用 起来 挺不错 的 。\n',
 '手机 使用 一天 了 ， 没有 什么 问题 ， 外观 非常 漂亮 ， 反应速度 都 很 好 。\n',
 '给 娃娃 卖 的 ， 娃 高兴 很\n',
 '用 起来 感觉 是 正品 ， 各 方面 都 不错 ， 暂时 没 发现 有 什么 问题 ， 值得 推荐\n',
 '物流 太快 了 ， 正品 无 问题\n',
 '是 行货 ， 手机 不错 ， 真真正正 的 裸机 ， 除了 充电器 和 耳机 没 别的 东西 ， 连 一个 手机 保护套 都 没有 ?\n',
 '前天 买 的 ， 今天 到 ！ 感觉 过 了 一

In [14]:
good_data=[clean(line) for line in good_data]

In [15]:
good_data

['整个 感觉 除了 送货 师傅 新手 来晚 了 外 其他 都 很 好',
 '京东 自营 很 不错',
 '外观 手感 使用 都 不错 好',
 '相信 这个 牌子 没有 验证 过 保修期',
 '好 售后服务 到位 能 解决 使用 中 遇见 的 问题',
 '你 哦 手机 用 着 挺 好 的 用 了 好长时间 才 过来 评价 挺不错 的',
 '给 老公 买 的 这 几天 用 的 挺 好 的 一直 在 京东 上 买 东西 很 可靠',
 '比 sSN 便宜 居然 SN 是 敏感 词语 真逗',
 '感觉 还好 没 发现 什么 问题 快递 很快 上午 下单 下午 就 到 就是 箱子 里面 没 放 气泡',
 '还 可以 的 用 起来 很 流畅',
 '非常 好 京东 配送 服务 好 老婆 很 喜欢 很 好 用',
 '查 了 不是 翻新机',
 '头 一次 买 苹果 开机 激活 费 了 点 事儿 不过 刚好 证明 机子 是 新 的 用 着 还 挺 好',
 '啥 都 好 就是 没 给 发票 你们 是不是 给 忘 了',
 '及时 速度 产品质量 佳',
 '可以 可以 可以 不 可以 可以 不 可以 可以 不 可以 可以 不 可以',
 '手机 不错 快递员 超有 礼貌 拆 包装 时 一直 强调 小心 不止 五星 的 好评',
 '梦寐以求 的 6s 终于 到手 了 用 起来 挺不错 的',
 '手机 使用 一天 了 没有 什么 问题 外观 非常 漂亮 反应速度 都 很 好',
 '给 娃娃 卖 的 娃 高兴 很',
 '用 起来 感觉 是 正品 各 方面 都 不错 暂时 没 发现 有 什么 问题 值得 推荐',
 '物流 太快 了 正品 无 问题',
 '是 行货 手机 不错 真真正正 的 裸机 除了 充电器 和 耳机 没 别的 东西 连 一个 手机 保护套 都 没有',
 '前天 买 的 今天 到 感觉 过 了 一个 世纪 那么 久 外包装 有点 压到 了 里面 的 塑料 枕头 坏 了 两个 不过 也 不 影响 总的来说 不错 了 快递 大哥 也 很 好 递 快递 的 时候 是 双手 递过来 的 很 有 礼貌 快 过年 了 大家 辛苦 了 给 快递 大哥 五星 好评 ╮ ╯ ▽ ╰ ╭',
 '刚刚开始 有些 小 插曲 后来 解决 了 现在 试用 中 电池 掉

In [27]:
# word_to_inx={}
# len(word_to_inx)

In [28]:
# word_to_inx['s'] = len(word_to_inx)

In [29]:
# word_to_inx

In [30]:
# len(word_to_inx)

In [32]:
#文本处理写成函数，对word先做labelencode，再对句子做labelencode，句子维度为MAX_LEN

#创建词典,用于word2index+index2word
word_to_inx={}
inx_to_word={}
#为数据生成label
def get_data():
    good_data=open('D:/opt/JD_nlp/data/good_cut_jieba.txt','r',encoding='utf-8').readlines()
    good_data=[clean(line) for line in good_data]
    good_data_label=[0 for i in range(len(good_data))]

    bad_data = open('D:/opt/JD_nlp/data/bad_cut_jieba.txt', 'r', encoding='utf-8').readlines()
    bad_data = [clean(line) for line in bad_data]
    bad_data_label = [1 for i in range(len(bad_data))]

    mid_data = open('D:/opt/JD_nlp/data/mid_cut_jieba.txt', 'r', encoding='utf-8').readlines()
    mid_data = [clean(line) for line in mid_data]
    mid_data_label = [2 for i in range(len(mid_data))]

    #total feature+label
    data=good_data+bad_data+mid_data
    data_label=good_data_label+bad_data_label+mid_data_label
    # print(data[0:5])
    # print(data_label[0:5])

    #创建一个word的set
    vocab=[word for s in data for word in s.split()]
    vocab=set(vocab)
    #print(vocab)

    for word in vocab:
        inx_to_word[len(word_to_inx)]=word
        word_to_inx[word]=len(word_to_inx)

    data_id=[]

    for sent in data:
        s_id=[]
        for word in sent.split():
            s_id.append(word_to_inx[word])
        #sent_labelincode 把sent补充为MAX_LEN
        s_id=s_id+[0]*(MAX_LEN-len(s_id))
        data_id.append(s_id)
        
    # 求出句子的最大长度
    # max_len=0
    # for s in data_id:
    #     if len(s)>max_len:
    #         max_len=len(s)
    # print(max_len)
    # 318
    # print(data_id[5:10])
    # print(len(data_id),' ',len(data_label))
    return data_id,data_label,word_to_inx,inx_to_word

In [39]:
#获取两个字典
def get_dic():
    _,_,word_to_inx,inx_to_word=get_data()
    return word_to_inx,inx_to_word

In [43]:
#将数据转化为tensor
def tensorFromData():
    #获取sent_labelencode和label
    data_id,data_lable,_,_=get_data()
    data_id_train, data_id_test, data_label_train, data_label_test = train_test_split(data_id, data_lable,
                                                                                      test_size=0.2,
                                                                                      random_state=233)
    data_id_train=torch.LongTensor(data_id_train)
    data_id_test=torch.LongTensor(data_id_test)
    data_label_train=torch.LongTensor(data_label_train)
    data_label_test=torch.LongTensor(data_label_test)
    return data_id_train,data_id_test,data_label_train,data_label_test
    # return data_id,data_lable

'''
pytroch创建自己的Dataset和Dataloader
https://blog.csdn.net/york1996/article/details/84141034
'''
class TextDataSet(Dataset):
    def __init__(self,inputs,outputs):
        self.inputs,self.outputs=inputs,outputs
    def __len__(self):
        return len(self.inputs)
    def __getitem__(self, index):
        return self.inputs[index],self.outputs[index]

In [45]:
import torch
from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
# import data_preprocess
import os

In [46]:
use_cuda=torch.cuda.is_available()

In [47]:
# 将数据划分为训练集和测试集
'''
DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, 
num_workers=0, collate_fn=default_collate, pin_memory=False, 
drop_last=False)

dataset：加载的数据集(Dataset对象)
batch_size：batch size
shuffle:：是否将数据打乱
sampler： 样本抽样，后续会详细介绍
num_workers：使用多进程加载的进程数，0代表不使用多进程
collate_fn： 如何将多个样本数据拼接成一个batch，一般使用默认的拼接方式即可
pin_memory：是否将数据保存在pin memory区，pin memory中的数据转到GPU会快一些
drop_last：dataset中的数据个数可能不是batch_size的整数倍，drop_last为True会将多出来不足一个batch的数据丢弃
'''
X_train, X_test, Y_train, Y_test = tensorFromData()
trainDataSet = TextDataSet(X_train, Y_train)
testDataSet = TextDataSet(X_test, Y_test)
trainDataLoader = DataLoader(trainDataSet, batch_size=16, shuffle=True)
testDataLoader = DataLoader(testDataSet, batch_size=16, shuffle=False)

In [49]:
# 获取字典
word_to_inx, inx_to_word = get_dic()
len_dic = len(word_to_inx)

In [50]:
# 定义超参数
MAXLEN = 64
input_dim = MAXLEN
emb_dim = 128
num_epoches = 20
batch_size = 16

In [51]:
#LSTM模型
'''
torch.nn是专门为神经网络设计的模块化接口。nn构建于autograd之上，可以用来定义和运行神经网络。
nn.Module是nn中十分重要的类,包含网络各层的定义及forward方法。
定义自已的网络：
    需要继承nn.Module类，并实现forward方法。
    一般把网络中具有可学习参数的层放在构造函数__init__()中，
    不具有可学习参数的层(如ReLU)可放在构造函数中，也可不放在构造函数中(而在forward中使用nn.functional来代替)
    
    只要在nn.Module的子类中定义了forward函数，backward函数就会被自动实现(利用Autograd)。
    在forward函数中可以使用任何Variable支持的函数，毕竟在整个pytorch构建的图中，是Variable在流动。还可以使用
    if,for,print,log等python语法.
    
    注：Pytorch基于nn.Module构建的模型中，只支持mini-batch的Variable输入方式，
    比如，只有一张输入图片，也需要变成 N x C x H x W 的形式：
    
    input_image = torch.FloatTensor(1, 28, 28)
    input_image = Variable(input_image)
    input_image = input_image.unsqueeze(0)   # 1 x 1 x 28 x 28

'''
'''
pytorch-embedding:
词嵌入。索引到这个表中,你需要使用 torch.LongTensor (索引为整数,不能为浮点数).
torch.nn.Embedding(m, n) 就可以了，m 表示单词的总数目，n 表示词嵌入的维度
# 定义词嵌入 
embeds = nn.Embedding(2, 5) # 2 个单词，维度 5 
# 得到词嵌入矩阵,开始是随机初始化的 
torch.manual_seed(1) embeds.weight 
# 输出结果： 
Parameter containing: 
-0.8923 -0.0583 -0.1955 -0.9656 0.4224 
0.2673 -0.4212 -0.5107 -1.5727 -0.1232 
[torch.FloatTensor of size 2x5]
'''
'''
pytorch-LSTM:
https://pytorch.org/docs/stable/nn.html?highlight=lstm#torch.nn.LSTM
input_size：x的特征维度
hidden_size：隐藏层的特征维度
num_layers：lstm隐层的层数，默认为1
bias：False则bih=0和bhh=0. 默认为True
batch_first：True则输入输出的数据格式为 (batch, seq, feature)
dropout：除最后一层，每一层的输出都进行dropout，默认为: 0
bidirectional：True则为双向lstm默认为False
输入：input, (h0, c0)
输出：output, (hn,cn)
'''
class LSTM_model(nn.Module):
    def __init__(self,len_dic,emb_dim):
        super(LSTM_model,self).__init__()
        #词嵌入
        self.embed=nn.Embedding(len_dic,emb_dim)  #b,64,128  -> 64,b,128
        #双层LSTM
        self.lstm1=nn.LSTM(input_size=emb_dim,hidden_size=256,dropout=0.2)#64,b,256
        self.lstm2=nn.LSTM(input_size=256,hidden_size=256,dropout=0.2)#64,b,256 -> b,256
        self.classify=nn.Linear(256,3)#b,3

    def forward(self, x):
        x=self.embed(x)
        # print(x.size())
        
        #https://blog.csdn.net/york1996/article/details/81876886
        # LSTM这里传进去的数据格式是[seq_len, batch_size, embedded_size]。
        # 而我们传进去的数据是[batch_size, seq_len]的样子，那经过embedding之后的结果是[batch_size, seq_len, embedded_size]。
        # 所以我们这里要将第二个维度和第一个维度做个调换。
        x=x.permute(1,0,2)
        
        out,_=self.lstm1(x)
        out,_=self.lstm2(out)
        out=out[-1,:,:]
        # print(out.size())
        #https://blog.csdn.net/york1996/article/details/81949843
        out=out.view(-1,256)
        out=self.classify(out)
        # print(out.size())
        return out

In [55]:
if use_cuda:
    model=LSTM_model(len_dic,emb_dim).cuda()
else:
    model=LSTM_model(len_dic,emb_dim)

C:\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [53]:
criterion = nn.CrossEntropyLoss()
optimzier = optim.Adam(model.parameters(), lr=1e-3)

In [54]:
best_acc=0
best_model=None

In [63]:
for epoch in range(num_epoches):# 数据集迭代 https://www.jianshu.com/p/043083d114d4
    #初始化loss和acc
    train_loss = 0
    train_acc = 0
#     通过 module.train() 和 module.eval() 来切换模型的 训练测试阶段。
    model.train()
    
    #steps print
    for i, data in enumerate(trainDataLoader):
        x, y = data
        if use_cuda:
            x, y = Variable(x).cuda(), Variable(y).cuda()
        else:
            x, y = Variable(x), Variable(y)
            
        # forward
        out = model(x)#预测
        loss = criterion(out, y)#计算loss
        
        train_loss += loss.item() * len(y)
        _, pre = torch.max(out, 1)
        num_acc = (pre == y).sum()
        train_acc += num_acc.item()
        # backward
        optimzier.zero_grad()
        loss.backward()
        optimzier.step()
        if (i + 1) % 100 == 0:#每100步计算精度
            print('[{}/{}],train loss is:{:.6f},train acc is:{:.6f}'.format(i, len(trainDataLoader),
                                                                            train_loss / (i * batch_size),
                                                                            train_acc / (i * batch_size)))
    print(
        'epoch:[{}],train loss is:{:.6f},train acc is:{:.6f}'.format(epoch,
                                                                     train_loss / (len(trainDataLoader) * batch_size),
                                                                     train_acc / (len(trainDataLoader) * batch_size)))
    #eval()时,框架会自动把BN和DropOut固定住,不会取平均,而是用训练好的值
    model.eval()
    eval_loss = 0
    eval_acc = 0
    for i, data in enumerate(testDataLoader):
        x, y = data
        if use_cuda:
            x = Variable(x, volatile=True).cuda()
            y = Variable(y, volatile=True).cuda()
        else:
            x = Variable(x, volatile=True)
            y = Variable(y, volatile=True)
        out = model(x)
        loss = criterion(out, y)
        eval_loss += loss.item() * len(y)
        _, pre = torch.max(out, 1)
        num_acc = (pre == y).sum()
        eval_acc += num_acc.item()
    print('test loss is:{:.6f},test acc is:{:.6f}'.format(
        eval_loss / (len(testDataLoader) * batch_size),
        eval_acc / (len(testDataLoader) * batch_size)))
    if best_acc<(eval_acc / (len(testDataLoader) * batch_size)):
        best_acc=eval_acc / (len(testDataLoader) * batch_size)
        best_model=model.state_dict()
        # print(best_model)
        print('best acc is {:.6f},best model is changed'.format(best_acc))

[99/786],train loss is:1.109198,train acc is:0.375000
[199/786],train loss is:1.103703,train acc is:0.374058
[299/786],train loss is:1.101802,train acc is:0.373328
[399/786],train loss is:1.100876,train acc is:0.371867
[499/786],train loss is:1.100285,train acc is:0.372745
[599/786],train loss is:1.099964,train acc is:0.369783
[699/786],train loss is:1.099732,train acc is:0.370351
epoch:[0],train loss is:1.096837,train acc is:0.370229


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


test loss is:1.094175,test acc is:0.367386
best acc is 0.367386,best model is changed
[99/786],train loss is:1.109234,train acc is:0.374369
[199/786],train loss is:1.103444,train acc is:0.378769
[299/786],train loss is:1.101684,train acc is:0.374373
[399/786],train loss is:1.100902,train acc is:0.371711
[499/786],train loss is:1.100384,train acc is:0.370366
[599/786],train loss is:1.100041,train acc is:0.369053
[699/786],train loss is:1.099751,train acc is:0.369814
epoch:[1],train loss is:1.096837,train acc is:0.370229
test loss is:1.094175,test acc is:0.367386
[99/786],train loss is:1.108807,train acc is:0.399621
[199/786],train loss is:1.103491,train acc is:0.385050
[299/786],train loss is:1.101700,train acc is:0.378972
[399/786],train loss is:1.100811,train acc is:0.374687
[499/786],train loss is:1.100224,train acc is:0.375125
[599/786],train loss is:1.099959,train acc is:0.372496
[699/786],train loss is:1.099698,train acc is:0.371334
epoch:[2],train loss is:1.096837,train acc is:0.

test loss is:1.094175,test acc is:0.367386
[99/786],train loss is:1.108922,train acc is:0.386364
[199/786],train loss is:1.103670,train acc is:0.372487
[299/786],train loss is:1.101745,train acc is:0.376254
[399/786],train loss is:1.100959,train acc is:0.369987
[499/786],train loss is:1.100378,train acc is:0.370491
[599/786],train loss is:1.099978,train acc is:0.371557
[699/786],train loss is:1.099762,train acc is:0.368652
epoch:[18],train loss is:1.096837,train acc is:0.370229
test loss is:1.094175,test acc is:0.367386
[99/786],train loss is:1.109179,train acc is:0.371843
[199/786],train loss is:1.103640,train acc is:0.367148
[299/786],train loss is:1.101846,train acc is:0.366639
[399/786],train loss is:1.100952,train acc is:0.366228
[499/786],train loss is:1.100354,train acc is:0.368863
[599/786],train loss is:1.100001,train acc is:0.369574
[699/786],train loss is:1.099733,train acc is:0.370082
epoch:[19],train loss is:1.096837,train acc is:0.370229
test loss is:1.094175,test acc is:

In [66]:
model_dir = "model/"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [69]:
os.getcwd()

'C:\\jupyter-notebook-files\\JD'

In [1]:
# torch.save(model.state_dict(),'./model/LSTM_model.pth')